In [1]:
import numpy as np
import pandas as pd
import transformers
# import nltk
import sklearn

C:\Users\v.veerati\.conda\envs\exam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import os
# os.add_dll_directory("C://ProgramData/Anaconda3/DLLs")

In [2]:
import torch
print(torch.cuda.is_available())

True


In [6]:
from datasets import load_dataset

dataset = load_dataset("FinanceInc/auditor_sentiment")

In [3]:
train_dataset= load_dataset("FinanceInc/auditor_sentiment", split='train')

Found cached dataset parquet (C:/Users/v.veerati/.cache/huggingface/datasets/FinanceInc___parquet/demo-org--auditor_review-deda15e5340bd334/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [4]:
test_dataset= load_dataset("FinanceInc/auditor_sentiment", split='test')

Found cached dataset parquet (C:/Users/v.veerati/.cache/huggingface/datasets/FinanceInc___parquet/demo-org--auditor_review-deda15e5340bd334/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
import numpy as np
print('avg token length for train set', np.average([len(x['sentence'].split(' ')) for x in train_dataset]))
print('avg token length for test set', np.average([len(x['sentence'].split(' ')) for x in test_dataset]))

avg token length for train set 23.08021666236781
avg token length for test set 23.227038183694532


# Convert dataset to dataframe 

In [6]:
def convert_to_df(dataset):
    sentences= []
    labels= []
    for each in dataset:
        sentences.append(each['sentence'])
        labels.append(each['label'])
    data= pd.DataFrame(sentences, columns=['text'])
    data['tag']= labels
    return data

In [7]:
train_df= convert_to_df(train_dataset)
test_df= convert_to_df(test_dataset)

In [8]:
train_df.head()

,text,tag
0,Altia 's operating profit jumped to EUR 47 mil...,2
1,The agreement was signed with Biohit Healthcar...,2
2,"Kesko pursues a strategy of healthy , focused ...",2
3,"Vaisala , headquartered in Helsinki in Finland...",1
4,"Also , a six-year historic analysis is provide...",1


In [9]:
from sklearn.model_selection import train_test_split 
train_df, dev_df = train_test_split(train_df, test_size=0.20, stratify=train_df['tag']) 

In [11]:
train_df.to_csv('./data/train.csv')
dev_df.to_csv('./data/dev.csv')
test_df.to_csv('./data/test.csv')

## Load back data

In [3]:
train_df = pd.read_csv('./data/train.csv')
dev_df = pd.read_csv('./data/dev.csv')
test_df = pd.read_csv('./data/test.csv')

In [4]:
train_df.tag.value_counts()

tag
1    1856
2     861
0     384
Name: count, dtype: int64

In [5]:
test_df.tag.value_counts()

tag
1    559
2    286
0    124
Name: count, dtype: int64

In [6]:
dev_df.tag.value_counts()

tag
1    464
2    216
0     96
Name: count, dtype: int64

In [7]:
train_df.head()

,Unnamed: 0,text,tag
0,756,"In the first half of 2008 , the Bank 's operat...",0
1,209,The solvency capital ratio of both companies w...,1
2,286,A CUT ABOVE Bring the outdoors in with these b...,1
3,3560,The above mentioned shareholders will suggest ...,1
4,2680,The operating margin of Aker Yards Cruise & Fe...,0


In [8]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

In [9]:
import torch

In [10]:
from torch.utils.data import DataLoader, Dataset

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels= 3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
text= train_df.iloc[0].text
tokens= tokenizer(text, padding=True, truncation=True, max_length= 128)
tokens

{'input_ids': [0, 1121, 5, 78, 457, 9, 2266, 2156, 5, 788, 128, 29, 1633, 1963, 1064, 7, 10353, 365, 4, 398, 475, 282, 31, 10353, 504, 4, 466, 475, 282, 2156, 150, 1161, 773, 1425, 1130, 7, 10353, 291, 4, 466, 475, 282, 31, 10353, 504, 4, 398, 475, 282, 11, 5, 78, 457, 9, 3010, 479, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
len(tokens['input_ids'])

57

In [15]:
class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_seq_length=128, tagged=True):
        ids= []
        labels=[]
        indicators= []
        for row in df.itertuples():
            text= row.text.strip()
            tokens= tokenizer.tokenize(text)
            if len(tokens)<= max_seq_length-2:
                indicators.append('+')
                input_ids= tokenizer.convert_tokens_to_ids(tokens)
                input_ids= tokenizer.build_inputs_with_special_tokens(input_ids)
                ids.append(input_ids)
                if tagged: 
                    labels.append(row.tag)
                else:
                    labels.append(-1)
            else:
                indicators.append('-')
        self.ids=ids
        self.labels=labels


    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.ids[idx],
            "labels": self.labels[idx]
        }

In [16]:
train_ds= SentimentDataset(train_df, tokenizer, 128, True)
dev_ds= SentimentDataset(dev_df, tokenizer, 128, True)
test_ds= SentimentDataset(test_df, tokenizer, 128, True)

In [17]:
len(train_ds)

3100

In [18]:
train_ds[0]

{'input_ids': [0,
  1121,
  5,
  78,
  457,
  9,
  2266,
  2156,
  5,
  788,
  128,
  29,
  1633,
  1963,
  1064,
  7,
  10353,
  365,
  4,
  398,
  475,
  282,
  31,
  10353,
  504,
  4,
  466,
  475,
  282,
  2156,
  150,
  1161,
  773,
  1425,
  1130,
  7,
  10353,
  291,
  4,
  466,
  475,
  282,
  31,
  10353,
  504,
  4,
  398,
  475,
  282,
  11,
  5,
  78,
  457,
  9,
  3010,
  479,
  2],
 'labels': 0}

In [19]:
def collate_fn(batch, max_len=128):
    max_len = max([len(f["input_ids"]) for f in batch])
    input_ids = [f["input_ids"]+[0]*(max_len-len(f["input_ids"])) for f in batch]
    input_mask = [[1.0]*len(f["input_ids"])+[0.0]*(max_len-len(f["input_ids"])) for f in batch]
    labels = [f['labels'] for f in batch]
    input_ids = torch.tensor(input_ids, dtype= torch.long)
    input_mask = torch.tensor(input_mask, dtype= torch.float)
    labels = torch.tensor(labels, dtype= torch.long)
    return{'input_ids':input_ids, 'attention_mask': input_mask, 'labels':labels}

In [20]:
for i in DataLoader(dev_ds, collate_fn=collate_fn, batch_size=5):
    print(i)

{'input_ids': tensor([[    0,   243,    16,    11,   516,    19,    63,   708,     7,  1056,
            15,  3919,   434,  5561,     8,  4615,  1902,  2365,  3184,   479,
             2,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0, 32811,  9925,  3146,   271,    40,    67,   535,   145,  2149,
            13, 12469,     8,  1309,  2156,  1078,     8,  1737,   289, 40111,
          8047,     9,  7746,   462, 10544,  2824,   479,     2,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0,   133, 13428,     9,     5,  4477,

{'input_ids': tensor([[    0,   133,   138,   473,    45,  9263,     5,  6797,    24,  5049,
            13,   479,     2,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,   487, 43946,    40,   535,     7,  3754,    11,   499,   709,
             9, 46603,  2156,     8, 16621,   493,    40,    28,  2149,    13,
          1861, 10765,     8,   518,   265,   479,     2,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
    

{'input_ids': tensor([[    0,  2118,  6725,  5945,   500,  2889,  2156,  9414,     4,  2156,
          1502,     4,   753,   480,   315,   532, 35720,   117,     4,   262,
             6,   398,  4540,     6, 32994,  2156,  1167,    15,  1502,     4,
           501,  2156,    21,  5530,     7, 10951,  1913,     4,    36, 11631,
          3036,  2156, 12587,  4839,   479,     2],
        [    0,  1121,  3010,  8923,   859, 11181,   431,    10, 10119,     9,
         10353, 11338,   479,   176,   475,     4,     2,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [    0, 39310,  2331,    26,    24,  2785,    24,    40,   146,  4522,
             9,   112,    12,   176,   475,  8502,   364,   710,    10,    76,
            30,  1353,  3009,    63,  1515,  1414,    23,    65,  1082,   479,
             

In [29]:
code2label= {0: 'Negative', 1: 'Neutral', 2: "Positive"}
labels=['Negative', 'Neutral', 'Positive']

## Training model

In [21]:
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Freezing all other layers and training only head

In [48]:
for param in model.base_model.parameters():
    param.requires_grad = False
    
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    predictions, labels= pred
    preds = np.argmax(predictions, axis= 1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [23]:
epochs= 8
batchsize= 16
len(train_ds)

3100

In [24]:
steps= epochs*len(train_ds)//batchsize
steps= steps/10

In [25]:
len(train_ds)

3100

In [26]:
steps

155.0

In [87]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args= TrainingArguments(output_dir= './output/roberat-base',
                                 evaluation_strategy= "steps",
                                 eval_steps= steps,
                                 save_steps= steps,
                                 num_train_epochs= epochs,
                                 per_device_train_batch_size= batchsize,
                                 per_device_eval_batch_size= batchsize,
                                 learning_rate= 2e-5,
                                 weight_decay= 0.01,
                                 adam_beta1=0.9,
                                 adam_beta2=.98,
                                 adam_epsilon=1e-06,
                                logging_steps= steps,
                                metric_for_best_model= 'f1',
                                load_best_model_at_end= True
                                )

In [88]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_ds,         # training dataset
    eval_dataset=dev_ds , # evaluation dataset
    compute_metrics=compute_metrics,
    callbacks= [EarlyStoppingCallback(early_stopping_patience=4)],
    data_collator= collate_fn
)

In [89]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
155,0.843500,0.856782,0.597938,0.597938,0.597938,0.597938
310,0.855600,0.852144,0.597938,0.597938,0.597938,0.597938
465,0.849200,0.848423,0.597938,0.597938,0.597938,0.597938
620,0.838300,0.845500,0.597938,0.597938,0.597938,0.597938
775,0.843400,0.842371,0.605670,0.605670,0.605670,0.605670
930,0.833800,0.840237,0.599227,0.599227,0.599227,0.599227
1085,0.847200,0.838376,0.610825,0.610825,0.610825,0.610825
1240,0.831700,0.837342,0.609536,0.609536,0.609536,0.609536
1395,0.827900,0.836801,0.609536,0.609536,0.609536,0.609536
1550,0.838300,0.836510,0.610825,0.610825,0.610825,0.610825


TrainOutput(global_step=1552, training_loss=0.8408934529294673, metrics={'train_runtime': 132.1635, 'train_samples_per_second': 187.646, 'train_steps_per_second': 11.743, 'total_flos': 748290448871280.0, 'train_loss': 0.8408934529294673, 'epoch': 8.0})

In [91]:
tokenizer.save_pretrained('./results/headonly/roberta-base')
trainer.save_model('./results/headonly/roberta-base')

### Eval on test set

## Load back the model

In [96]:
model= AutoModelForSequenceClassification.from_pretrained('./results/headonly/roberta-base', num_labels= 3)
model.to(device)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [103]:
from tqdm import tqdm
def predict(ds, model, batchsize, device):
    dataloader= DataLoader(ds, batch_size= batchsize, collate_fn= collate_fn, drop_last= False)
    labels, predictions, probabilities= [], [], []
    with tqdm(total= len(dataloader)) as pbar:
        for i, batch in enumerate(dataloader):
            inputs={'input_ids':batch['input_ids'].to(device),
                   'attention_mask': batch['attention_mask'].to(device)}
            labels += batch['labels'].tolist()
            with torch.no_grad():
                logit= model(**inputs)[0]
#                 probabilities.extend(logit.softmax(dim=-1).detach.cpu().numpy.tolist())
                pred= torch.argmax(logit, dim=-1)
            predictions += pred.tolist()
            pbar.update()
    return labels, predictions
            

In [104]:
yy_true, yy_pred= predict(test_ds, model, 16, device)

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:03<00:00, 16.66it/s]


In [114]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
def eval(yy_true, yy_pred, labels, code2label, average= 'micro'):
    acc= np.round(accuracy_score(yy_true, yy_pred)*100, 1)
    F1_score= np.round(f1_score(yy_true, yy_pred, average=average)*100, 1)
    print('Accuracy Score:', acc)
    print('F1 Score:', F1_score)
    yy_pred= [code2label[x] for x in yy_pred]
    yy_true = [code2label[x] for x in yy_true]
    report= classification_report(yy_true, yy_pred, labels=labels, digits=3)
    print(report)

In [116]:
eval(yy_true, yy_pred, labels,code2label)

Accuracy Score: 59.1
F1 Score: 59.1
              precision    recall  f1-score   support

    Negative      0.000     0.000     0.000       124
     Neutral      0.592     0.998     0.743       559
    Positive      0.577     0.052     0.096       286

    accuracy                          0.591       969
   macro avg      0.390     0.350     0.280       969
weighted avg      0.512     0.591     0.457       969



C:\Users\v.veerati\.conda\envs\exam\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\v.veerati\.conda\envs\exam\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\v.veerati\.conda\envs\exam\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Finetune last 3 layers

In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels= 3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
model.id2label= code2label
model.label2id= {y:x for x, y in code2label.items()}

In [33]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1476, -0.0365,  0.0753,  ..., -0.0023,  0.0172, -0.0016],
        [ 0.0156,  0.0076, -0.0118,  ..., -0.0022,  0.0081, -0.0156],
        [-0.0347, -0.0873, -0.0180,  ...,  0.1174, -0.0098, -0.0355],
        ...,
        [ 0.0304,  0.0504, -0.0307,  ...,  0.0377,  0.0096,  0.0084],
        [ 0.0623, -0.0596,  0.0307,  ..., -0.0920,  0.1080, -0.0183],
        [ 0.1259, -0.0145,  0.0332,  ...,  0.0121,  0.0342,  0.0168]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0115,  0.0204,  0.0197,  ...,  0.0050, -0.0274, -0.0439],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0346, -0.0169, -0.0895,  ..., -0.0542,  0.0291,  0.0173],
        ...,
        [ 0.1191, -0.0587, -0.0396,  ..., -0.0179,  0.1249,  0.0205],
        [ 0.0833, -0.0272, -0.0368,  ..., -0.0429,  0.1288, -0.0052],
        [ 0.0850,  0.2573, -0.1257,  ..., -0.0952,  0.1443, -0.0131]],
       requires_grad=True)
Parameter containing:
tensor([[0.,

Parameter containing:
tensor([[ 0.0512,  0.0333,  0.0150,  ..., -0.0113, -0.0341, -0.0767],
        [-0.0290,  0.0225, -0.0138,  ..., -0.0399, -0.0560,  0.0398],
        [-0.0351, -0.0066,  0.0213,  ...,  0.0908, -0.0063,  0.0688],
        ...,
        [-0.0522, -0.0978,  0.0016,  ...,  0.0445,  0.0494, -0.0648],
        [-0.0149, -0.0285, -0.0291,  ...,  0.0429, -0.0368,  0.0067],
        [-0.0731, -0.0401, -0.0239,  ..., -0.0797, -0.0129, -0.0376]],
       requires_grad=True)
Parameter containing:
tensor([-3.6640e-03, -5.6725e-03, -1.4206e-02,  3.3226e-03, -2.6684e-03,
         1.4702e-02, -2.1591e-03, -9.7656e-03, -2.5742e-02, -4.1223e-04,
         1.3618e-03,  6.5384e-03, -9.8267e-03,  2.4910e-03, -1.9287e-02,
        -6.9466e-03, -1.0605e-03, -1.3435e-02,  1.5327e-02, -1.4336e-02,
        -1.4198e-02, -9.9335e-03, -6.7024e-03,  3.9825e-03,  1.0666e-02,
        -4.4250e-03, -1.5764e-03,  2.3651e-02, -2.3689e-03, -2.3468e-02,
         1.2711e-02, -3.6545e-03, -7.3929e-03,  1.4931e-0

Parameter containing:
tensor([ 6.9275e-02,  1.0486e-01, -1.4725e-02, -3.2532e-02, -2.8809e-02,
         5.2261e-03,  8.6121e-02, -7.2784e-03, -1.9638e-02, -1.2260e-02,
         9.5367e-03,  3.1952e-02, -1.2976e-01, -1.3245e-01,  3.7445e-02,
        -3.3905e-02, -3.9246e-02,  3.4851e-02,  1.7664e-01, -4.8340e-02,
         6.0005e-03, -7.9834e-02, -2.3865e-02, -2.8458e-02,  5.6854e-02,
        -7.5806e-02,  1.7195e-03,  5.6458e-03, -7.9834e-02, -2.4567e-02,
        -1.2067e-01, -1.6846e-01,  7.4272e-03, -2.1027e-02,  5.7373e-03,
         1.1340e-01,  1.5083e-02,  7.5340e-03, -1.3452e-01,  9.8694e-02,
         1.3135e-01,  1.6041e-03,  7.3669e-02, -9.3613e-03,  1.0330e-02,
         2.5864e-03, -8.5327e-02,  5.5542e-02, -4.8859e-02, -2.5696e-02,
        -1.2337e-02,  1.0852e-01,  9.1919e-02, -5.0873e-02,  4.2480e-02,
        -3.7354e-02, -7.1899e-02, -1.5594e-02,  6.0547e-02,  2.2293e-02,
         4.1779e-02, -6.2256e-02,  1.6968e-02,  3.4119e-02,  7.4692e-03,
        -4.6936e-02,  1.6617e

In [34]:
# Freeze the last 3 layers
for param in model.parameters():
    param.requires_grad = True  # Ensure all parameters initially have gradients

# Count the total number of layers
total_layers = len(list(model.parameters()))

# Identify the layers to freeze (e.g., last 3 layers)
layers_to_freeze = 3

# Iterate through the layers in reverse order and freeze them
for idx, param in enumerate(model.parameters()):
    if idx >= total_layers - layers_to_freeze:
        param.requires_grad = False

In [35]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1476, -0.0365,  0.0753,  ..., -0.0023,  0.0172, -0.0016],
        [ 0.0156,  0.0076, -0.0118,  ..., -0.0022,  0.0081, -0.0156],
        [-0.0347, -0.0873, -0.0180,  ...,  0.1174, -0.0098, -0.0355],
        ...,
        [ 0.0304,  0.0504, -0.0307,  ...,  0.0377,  0.0096,  0.0084],
        [ 0.0623, -0.0596,  0.0307,  ..., -0.0920,  0.1080, -0.0183],
        [ 0.1259, -0.0145,  0.0332,  ...,  0.0121,  0.0342,  0.0168]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0115,  0.0204,  0.0197,  ...,  0.0050, -0.0274, -0.0439],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0346, -0.0169, -0.0895,  ..., -0.0542,  0.0291,  0.0173],
        ...,
        [ 0.1191, -0.0587, -0.0396,  ..., -0.0179,  0.1249,  0.0205],
        [ 0.0833, -0.0272, -0.0368,  ..., -0.0429,  0.1288, -0.0052],
        [ 0.0850,  0.2573, -0.1257,  ..., -0.0952,  0.1443, -0.0131]],
       requires_grad=True)
Parameter containing:
tensor([[0.,

Parameter containing:
tensor([[ 0.0512,  0.0333,  0.0150,  ..., -0.0113, -0.0341, -0.0767],
        [-0.0290,  0.0225, -0.0138,  ..., -0.0399, -0.0560,  0.0398],
        [-0.0351, -0.0066,  0.0213,  ...,  0.0908, -0.0063,  0.0688],
        ...,
        [-0.0522, -0.0978,  0.0016,  ...,  0.0445,  0.0494, -0.0648],
        [-0.0149, -0.0285, -0.0291,  ...,  0.0429, -0.0368,  0.0067],
        [-0.0731, -0.0401, -0.0239,  ..., -0.0797, -0.0129, -0.0376]],
       requires_grad=True)
Parameter containing:
tensor([-3.6640e-03, -5.6725e-03, -1.4206e-02,  3.3226e-03, -2.6684e-03,
         1.4702e-02, -2.1591e-03, -9.7656e-03, -2.5742e-02, -4.1223e-04,
         1.3618e-03,  6.5384e-03, -9.8267e-03,  2.4910e-03, -1.9287e-02,
        -6.9466e-03, -1.0605e-03, -1.3435e-02,  1.5327e-02, -1.4336e-02,
        -1.4198e-02, -9.9335e-03, -6.7024e-03,  3.9825e-03,  1.0666e-02,
        -4.4250e-03, -1.5764e-03,  2.3651e-02, -2.3689e-03, -2.3468e-02,
         1.2711e-02, -3.6545e-03, -7.3929e-03,  1.4931e-0

Parameter containing:
tensor([0.6577, 0.8452, 0.7383, 0.8882, 0.8960, 0.8579, 0.7500, 0.8203, 0.7354,
        0.7939, 0.8555, 0.8931, 0.7788, 0.9282, 0.8496, 0.8345, 0.8726, 0.8623,
        0.6987, 0.9458, 0.7334, 0.8154, 0.7808, 0.8174, 0.8799, 0.7925, 0.8335,
        0.7642, 0.8433, 0.9189, 0.7783, 0.8511, 0.7681, 0.7930, 0.8633, 0.7588,
        0.7725, 0.7451, 0.9004, 0.7827, 0.8965, 0.8975, 0.7886, 0.7617, 0.6973,
        0.7505, 0.9326, 0.8374, 0.6836, 0.8433, 0.8560, 0.7173, 0.7202, 0.8135,
        0.7559, 0.8745, 0.7012, 0.8887, 0.7871, 0.7715, 0.8721, 0.9932, 0.8374,
        0.9150, 0.8193, 0.8003, 0.7314, 0.9150, 0.7134, 0.7798, 0.7446, 0.8403,
        0.7505, 0.8813, 0.7026, 0.8384, 0.8228, 1.0000, 0.8647, 0.8960, 0.7729,
        0.8364, 0.9995, 0.8760, 0.8735, 0.8154, 0.6729, 0.9048, 0.7329, 0.7305,
        0.8901, 0.7974, 0.7646, 0.9053, 0.7578, 0.9087, 0.7656, 0.9971, 0.6509,
        0.9077, 0.8853, 0.7827, 0.8359, 0.7554, 0.7715, 0.9409, 0.8462, 0.8237,
        0.7295, 0.

In [36]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args= TrainingArguments(output_dir= './output/roberta-base',
                                 evaluation_strategy= "steps",
                                 eval_steps= steps,
                                 save_steps= steps,
                                 num_train_epochs= epochs,
                                 per_device_train_batch_size= batchsize,
                                 per_device_eval_batch_size= batchsize,
                                 learning_rate= 2e-5,
                                 weight_decay= 0.01,
                                 adam_beta1=0.9,
                                 adam_beta2=.98,
                                 adam_epsilon=1e-06,
                                logging_steps= steps,
                                metric_for_best_model= 'f1',
                                load_best_model_at_end= True
                                )

In [37]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_ds,         # training dataset
    eval_dataset=dev_ds , # evaluation dataset
    compute_metrics=compute_metrics,
    callbacks= [EarlyStoppingCallback(early_stopping_patience=4)],
    data_collator= collate_fn
)

In [38]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
155,0.652000,0.373921,0.847938,0.847938,0.847938,0.847938
310,0.359500,0.400636,0.842784,0.842784,0.842784,0.842784
465,0.262000,0.377469,0.875000,0.875000,0.875000,0.875000
620,0.198000,0.649905,0.835052,0.835052,0.835052,0.835052
775,0.158200,0.623837,0.854381,0.854381,0.854381,0.854381
930,0.116100,0.788771,0.847938,0.847938,0.847938,0.847938
1085,0.074200,0.693899,0.878866,0.878866,0.878866,0.878866
1240,0.060100,0.802117,0.864691,0.864691,0.864691,0.864691
1395,0.050400,0.804750,0.868557,0.868557,0.868557,0.868557
1550,0.018700,0.807790,0.878866,0.878866,0.878866,0.878866


TrainOutput(global_step=1552, training_loss=0.19467255552624863, metrics={'train_runtime': 409.5514, 'train_samples_per_second': 60.554, 'train_steps_per_second': 3.79, 'total_flos': 748290448871280.0, 'train_loss': 0.19467255552624863, 'epoch': 8.0})

In [39]:
tokenizer.save_pretrained('./results/last3layers/roberta-base')
trainer.save_model('./results/last3layers/roberta-base')

### Eval on test set

## Load back the model

In [40]:
model= AutoModelForSequenceClassification.from_pretrained('./results/last3layers/roberta-base', num_labels= 3)
model.to(device)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

# Evaluation of model

In [36]:
from tqdm import tqdm
def predict(ds, model, batchsize, device):
    dataloader= DataLoader(ds, batch_size= batchsize, collate_fn= collate_fn, drop_last= False)
    labels, predictions, probabilities= [], [], []
    with tqdm(total= len(dataloader)) as pbar:
        for i, batch in enumerate(dataloader):
            inputs={'input_ids':batch['input_ids'].to(device),
                   'attention_mask': batch['attention_mask'].to(device)}
            labels += batch['labels'].tolist()
            with torch.no_grad():
                logit= model(**inputs)[0]
#                 probabilities.extend(logit.softmax(dim=-1).detach.cpu().numpy.tolist())
                pred= torch.argmax(logit, dim=-1)
            predictions += pred.tolist()
            pbar.update()
    return labels, predictions
            

In [37]:
yy_true, yy_pred= predict(test_ds, model, 16, device)

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:03<00:00, 16.54it/s]


In [38]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
def eval(yy_true, yy_pred, labels, code2label, average= 'micro'):
    acc= np.round(accuracy_score(yy_true, yy_pred)*100, 1)
    F1_score= np.round(f1_score(yy_true, yy_pred, average=average)*100, 1)
    print('Accuracy Score:', acc)
    print('F1 Score:', F1_score)
    yy_pred= [code2label[x] for x in yy_pred]
    yy_true = [code2label[x] for x in yy_true]
    report= classification_report(yy_true, yy_pred, labels=labels, digits=3)
    print(report)

In [39]:
code2label= {0: 'Negative', 1: 'Neutral', 2: "Positive"}
labels=['Negative', 'Neutral', 'Positive']

In [40]:
eval(yy_true, yy_pred, labels,code2label)

Accuracy Score: 88.5
F1 Score: 88.5
              precision    recall  f1-score   support

    Negative      0.850     0.911     0.879       124
     Neutral      0.905     0.905     0.905       559
    Positive      0.863     0.836     0.849       286

    accuracy                          0.885       969
   macro avg      0.873     0.884     0.878       969
weighted avg      0.886     0.885     0.885       969



# Creating pipelines 

In [41]:
code2label

{0: 'Negative', 1: 'Neutral', 2: 'Positive'}

In [25]:
from transformers import pipeline
from torch.utils.data import Dataset
from tqdm.auto import tqdm

pipe = pipeline("text-classification",  model= './results/last3layers/roberta-base' , tokenizer= './results/last3layers/roberta-base', device=device)

In [26]:
texts= [x.text for x in test_df.iloc[:10].itertuples()]
pipe(texts)

[{'label': 'LABEL_2', 'score': 0.7922744154930115},
 {'label': 'LABEL_2', 'score': 0.871820330619812},
 {'label': 'LABEL_2', 'score': 0.9599653482437134},
 {'label': 'LABEL_2', 'score': 0.8572084307670593},
 {'label': 'LABEL_2', 'score': 0.9560665488243103},
 {'label': 'LABEL_2', 'score': 0.9731866121292114},
 {'label': 'LABEL_2', 'score': 0.8645796179771423},
 {'label': 'LABEL_2', 'score': 0.95745849609375},
 {'label': 'LABEL_2', 'score': 0.9631744623184204},
 {'label': 'LABEL_1', 'score': 0.7068823575973511}]